# MIMIC-IV query API tutorial

This notebook shows examples of how to use the cyclops.query API on [MIMIC-IV v2.0](https://physionet.org/content/mimiciv/2.0/).

* First, setup the MIMIC-IV database according to the instructions in [mimic-code](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iv/buildmimic/postgres).
* The database is assumed to be hosted using postgres. Update the config parameters such as username and password, passed to `MIMICIVQuerier` accordingly.

## Imports and instantiate `MIMICIVQuerier`

In [1]:
from cyclops.query import MIMICIVQuerier
from cyclops.query.ops import ConditionRegexMatch, ConditionStartsWith

mimic = MIMICIVQuerier(
    dbms="postgresql",
    port=5432,
    host="localhost",
    database="mimiciv-2.0",
    user="postgres",
    password="pwd",
)

2023-01-16 12:34:11,469 INFO cyclops.query.orm - Database setup, ready to run queries!


## Example 1. Get all patient encounters from 2015 (approximate year of care).

In [2]:
encounters = mimic.patient_encounters(years=2015)
encounters.run()
print(f"{len(encounters.data)} rows extracted!")

2023-01-16 12:34:12,619 INFO cyclops.query.orm - Query returned successfully!
2023-01-16 12:34:12,620 INFO cyclops.utils.profile - Finished executing function run_query in 1.107362 s


69460 rows extracted!


## Example 2. Get all patient encounters with diagnoses (`schizophrenia` in ICD long title), in the year 2015.

In [3]:
encounters = mimic.patient_encounters(years=[2015])
encounters_schizophrenia = mimic.patient_diagnoses(
    diagnosis_substring="schizophrenia", patients=encounters
)
encounters_schizophrenia.run()
print(f"{len(encounters_schizophrenia.data)} rows extracted!")
encounters_schizophrenia.data["diagnosis_title"].value_counts()

2023-01-16 12:34:13,405 INFO cyclops.query.orm - Query returned successfully!
2023-01-16 12:34:13,406 INFO cyclops.utils.profile - Finished executing function run_query in 0.756868 s


4731 rows extracted!


Unspecified schizophrenia, unspecified                                     1511
Schizophrenia, unspecified                                                 1431
Paranoid type schizophrenia, unspecified                                    527
Paranoid schizophrenia                                                      381
Paranoid type schizophrenia, chronic                                        279
Paranoid type schizophrenia, chronic with acute exacerbation                174
Unspecified schizophrenia, chronic                                           77
Catatonic schizophrenia                                                      68
Other specified types of schizophrenia, unspecified                          43
Catatonic type schizophrenia, unspecified                                    37
Other schizophrenia                                                          33
Disorganized type schizophrenia, chronic with acute exacerbation             20
Catatonic type schizophrenia, chronic wi

## Example 3. Advanced - uses `ConditionRegexMatch` from `cyclops.query.ops`. Get all patient encounters with diagnoses (ICD long title contains `schizophrenia` and `chronic` ), in the year 2015.

In [4]:
encounters = mimic.patient_encounters(years=[2015])
diagnoses = mimic.patient_diagnoses(patients=encounters)
patients_schizophrenia_chronic = ConditionRegexMatch(
    "diagnosis_title", r"(?=.*schizophrenia)(?=.*chronic)"
)(diagnoses.query)
data = mimic.get_interface(patients_schizophrenia_chronic).run()
print(f"{len(data)} rows extracted!")

2023-01-16 12:34:14,681 INFO cyclops.query.orm - Query returned successfully!
2023-01-16 12:34:14,682 INFO cyclops.utils.profile - Finished executing function run_query in 1.248306 s


644 rows extracted!


## Example 4. Advanced - uses `ConditionStartsWith` from cyclops.query.ops. Get all patient encounters with diagnoses (starts with `Paranoid` in ICD long title), in the year 2015.

In [5]:
encounters = mimic.patient_encounters(years=[2015])
diagnoses = mimic.patient_diagnoses(patients=encounters)
patients_paranoia = ConditionStartsWith("diagnosis_title", "Paranoid")(diagnoses.query)
data = mimic.get_interface(patients_paranoia).run()
print(f"{len(data)} rows extracted!")

2023-01-16 12:34:24,056 INFO cyclops.query.orm - Query returned successfully!
2023-01-16 12:34:24,057 INFO cyclops.utils.profile - Finished executing function run_query in 9.342424 s


1429 rows extracted!


## Example 5. Get all patient encounters with diagnoses (ICD code is F209).

In [6]:
patients = mimic.patients()
patients_f209 = mimic.patient_diagnoses(diagnosis_codes=["F209"], patients=patients)
patients_f209.run()
print(f"{len(patients_f209.data)} rows extracted!")

2023-01-16 12:34:32,477 INFO cyclops.query.orm - Query returned successfully!
2023-01-16 12:34:32,478 INFO cyclops.utils.profile - Finished executing function run_query in 8.393982 s


1431 rows extracted!


## Example 6. Get all patient encounters with diagnoses (`delirium` in ICD long title).

In [7]:
patients = mimic.patients()
patients_delirium = mimic.patient_diagnoses(
    diagnosis_substring="delirium", patients=patients
)
patients_delirium.run()
print(f"{len(patients_delirium.data)} rows extracted!")

2023-01-16 12:34:33,139 INFO cyclops.query.orm - Query returned successfully!
2023-01-16 12:34:33,140 INFO cyclops.utils.profile - Finished executing function run_query in 0.645764 s


9000 rows extracted!


In [8]:
patients_delirium.data["diagnosis_title"].value_counts()

Delirium due to conditions classified elsewhere                                  3938
Delirium due to known physiological condition                                    3075
Drug-induced delirium                                                             963
Alcohol withdrawal delirium                                                       373
Alcohol dependence with withdrawal delirium                                       210
Vascular dementia, with delirium                                                  169
Senile dementia with delirium                                                      88
Subacute delirium                                                                  35
Alcohol dependence with intoxication delirium                                      28
Alcohol abuse with intoxication delirium                                           20
Other psychoactive substance use, unspecified with intoxication with delirium      15
Sedative, hypnotic or anxiolytic dependence with withd

## Example 7. Get routine vital signs for patients from year 2015, limit to 100 rows.

In [9]:
encounters = mimic.patient_encounters(years=[2018])
patients_vitals = mimic.events(
    patient_encounters_table=encounters.query, categories="Routine Vital Signs"
)
patients_vitals.run(limit=100)
print(f"{len(patients_vitals.data)} rows extracted!")

2023-01-16 12:36:09,988 INFO cyclops.query.orm - Query returned successfully!
2023-01-16 12:36:09,990 INFO cyclops.utils.profile - Finished executing function run_query in 96.810362 s


100 rows extracted!


## Example 8. Get hemoglobin lab tests for patients from year 2009, limit to 100 rows.

In [10]:
encounters = mimic.patient_encounters(years=[2009])
patients_hemo_labs = mimic.events(
    categories="labs",
    patient_encounters_table=encounters.query,
    event_names="hemoglobin",
)
patients_hemo_labs.run(limit=100)
print(f"{len(patients_hemo_labs.data)} rows extracted!")

2023-01-16 12:36:11,696 INFO cyclops.query.orm - Query returned successfully!
2023-01-16 12:36:11,698 INFO cyclops.utils.profile - Finished executing function run_query in 1.677006 s


100 rows extracted!


## Example 9. Get all lab events for patients from year 2010, that match substring "sodium", limit to 100 rows.

In [11]:
encounters = mimic.patient_encounters(years=[2009])
patients_sodium_labs = mimic.events(
    categories="labs",
    patient_encounters_table=encounters.query,
    event_name_substring="sodium",
)
patients_sodium_labs.run(limit=100)
print(f"{len(patients_sodium_labs.data)} rows extracted!")

2023-01-16 12:36:14,289 INFO cyclops.query.orm - Query returned successfully!
2023-01-16 12:36:14,290 INFO cyclops.utils.profile - Finished executing function run_query in 2.549300 s


100 rows extracted!


## Example 10. Get respiratory events for patients from year 2009, limit to 100 rows.

In [12]:
encounters = mimic.patient_encounters(years=[2009])
patients_respiratory = mimic.events(
    categories="Respiratory", patient_encounters_table=encounters.query
)
patients_respiratory.run(limit=100)
print(f"{len(patients_respiratory.data)} rows extracted!")

2023-01-16 12:37:35,834 INFO cyclops.query.orm - Query returned successfully!
2023-01-16 12:37:35,836 INFO cyclops.utils.profile - Finished executing function run_query in 81.502515 s


100 rows extracted!


## Example 11. Get heart rate measurements of patients with delirium diagnoses, limit to 100 rows.

In [13]:
patients = mimic.patients()
encounters = mimic.patient_encounters(patients_table=patients.query)
delirium_encounters = mimic.patient_diagnoses(
    diagnosis_substring="delirium", patients_table=encounters.query
)
patients_delirium_heart_rate = mimic.events(
    event_names="Heart Rate", patient_encounters_table=delirium_encounters.query
)
patients_delirium_heart_rate.run(limit=100)
print(f"{len(patients_delirium_heart_rate.data)} rows extracted!")

2023-01-16 12:38:50,455 INFO cyclops.query.orm - Query returned successfully!
2023-01-16 12:38:50,457 INFO cyclops.utils.profile - Finished executing function run_query in 74.580848 s


100 rows extracted!


## Example 12. Get d_items dimension table, for quick reference.

In [14]:
event_lookup_table = mimic.get_interface(mimic.get_table("event_labels"))
lookup_data = event_lookup_table.run()
print(f"{len(lookup_data)} rows extracted!")

2023-01-16 12:38:50,497 INFO cyclops.query.orm - Query returned successfully!
2023-01-16 12:38:50,498 INFO cyclops.utils.profile - Finished executing function run_query in 0.030484 s


4014 rows extracted!


## Example 13. Get all female patient encounters from year 2015, and return as dask dataframe (lazy evaluation) with 4 partitions (batches).

In [15]:
encounters = mimic.patient_encounters(years=2015, sex="F")
encounters.run(backend="dask", index_col="encounter_id", n_partitions=4)
print(f"{len(encounters.data)} rows extracted!")
print(f"Return type: {type(encounters.data)}")
print(f"Number of partitions: {encounters.data.npartitions}")

2023-01-16 12:38:51,143 INFO cyclops.query.orm - Query returned successfully!
2023-01-16 12:38:51,143 INFO cyclops.utils.profile - Finished executing function run_query in 0.618330 s


35639 rows extracted!
Return type: <class 'dask.dataframe.core.DataFrame'>
Number of partitions: 4


## Example 14. Running a raw SQL string.

In [16]:
data = mimic._db.run_query("SELECT * FROM mimiciv_hosp.admissions LIMIT 100")
print(f"{len(data)} rows extracted!")

2023-01-16 12:38:52,854 INFO cyclops.query.orm - Query returned successfully!
2023-01-16 12:38:52,854 INFO cyclops.utils.profile - Finished executing function run_query in 0.008507 s


100 rows extracted!
